In [ ]:
# 강의 url : https://youtube.com/playlist?list=PL9mhQYIlKEhdrYpsGk8X4qj3tQUuaDhrl
# Colab 원본 : https://colab.research.google.com/drive/1wMRLEK0cwToY0yCMW4K8kkP2dHS5KOtq?usp=sharing#scrollTo=TpycP9ofzGek
# 참조 논문 : Deep Speech to, Listen attend and spell
# 딥러닝 기초 리뷰 : FC Layer, CNN, RNN, LSTM, Attention
# Audio Classfication & Tagging : 데이터 파이프라인의 이해
# CTC : 논문의 이해, CTC Loss 쓰는 법, Deep Speech2 구현
# LAS : Extra 모델 아키텍쳐 이해

In [ ]:
# Spectrogram 을 input 으로 받고, invariant convolution 을 거친 다음에, RNN (Recurrent Neural Network) 을 거치고, Fully Connected 를 거친다.
# 마지막으로 나온 CTC(Connectionist Temporal Classification) 를 Decode 하여 결과값을 얻어냄.
# IO Module, Network Architecture, Loss 함수만 잘 설정하면 좋은 모델 만들 수 있음.

In [ ]:
# Connectivity Patterns + Nonlinearity modules

# Connectivity Patterns
# - Fully-Connected (FC Layer)
# - Convolutional
# - Dilated
# - Recurrent
# - Skip / Residual
# - Random

# Nonlinearity modules
# - ReLU
# - Sigmoid
# - Tanh
# - GRU
# - LSTM

# Loss
# - Cross Entropy
# - Adversarial
# - Variational
# - Maximum Likelihood
# - L1 and L2

# Optimizer
# - SGD
# - Momentum
# - RMSProp
# - Adagrad
# - Adam
# - Second Order

# Hyper Parameters
# - Learning rate
# - Weight decay
# - Layer size
# - Batch size
# - Dropout rate
# - Weight initialization
# - Data augmentation
# - Gradient clipping
# - Momentum

In [ ]:
# Fully-Connected (FC Layer) 는 Multi-Layer Perceptron 에 Non-Linearity module 만 뺀 것
# x 에 대해 모든 입력값을 가중치 W 입력을 통해 z1, activation 함수를 통해 h, 또 다른 activation 함수로 z2 (y)
# h 를 만드는 Non-Linearity Function 으로는 Sigmoid, Tanh, ReLU, Leaky ReLU 가 있다.

In [ ]:
# Convolution Layer 는 입력을 필터로 사용하여 Convolution 연산 진행
# 하이퍼 파라미터에는 필터 크기 F 및 보폭 S 포함. 결과 출력 O 를 feature map, activation map 이라고 부름
# Pooling Layer 는 다운 샘플링 작업으로, 일반적으로 Convolution Layer 이후 적용, spatial invariance (공간에 영향 없는) 를 수행.
# 특히, Max pool 과 Average Pool 는 각각 최대 값과 평균값을 취하는 특수 종류의 풀링
# Pooling 된 데이터를 Fully Connected 로 연결시킬 수 있음

In [ ]:
# 1-D CNN in Spectrogram
# Convolution filter 의 크기가 frequency 영역대는 고정되어 있으며, Time 에 따라 진행

# 2-D CNN in Spectrogram
# Convolution filter 의 크기가 frequency과 Time에 따라서 진행됩니다.

# Sample CNN
# 바로 input 데이터를 waveform 그 자체로 사용할 수 있다.
# CNN 이 "phase-invariant" representation 을 반영

In [ ]:
# RNN 은 Hidden State 를 유지하면서 이전 출력을 입력으로 사용할 수 있는 신경망임.
# a^<T> = tanh(WaxX^<t> + WaaA^<t-1> + ba)
# y^<T> = softmax(WyaA^<t> + by)

# LSTM
# GRU (Gated Recurrent Unit) 및 LSTM (Long Short-Term Memory Unit)은 기존 RNN에서 vanishing gradient 를 처리하며 LSTM은 GRU의 일반화된 모델임.
# Forget 게이트 등 여러 게이트를 바탕으로 옛 데이터를 기억하도록 함 (RNN 개선)

In [ ]:
# Attention
# Encoding 된 정보와 Decoding 된 정보의 Hidden State 간의 Alignment 를 맞추는 것
# Hidden State 를 맞추다보면 어떤 스텝에서 어떤 Utterance 를 나타내는지 알 수 있음.
# 어떤 시점에 대한 인식과 번역에 쓰임.

# RNN계열 encoding 방식에서는 계속 마지막 hidden state까지 학습을 하면서 연산을 해야 했다.
# 이러한 문제를 해결하기 위해 input source 와 hidden state 의 관계를 학습시키는 추가적인 Network 를 만든 Attention 이 나옴.
# 이 attention 은 output 에 의하여 weight 를 학습하게 된다.

In [1]:
# FC Layer

layer_dims = [5,4,3,1]

import numpy as np

def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    cache = Z
    return A, cache

def relu(Z):
    A = np.maximum(0,Z)
    assert(A.shape == Z.shape)
    cache = Z
    return A, cache

def initialize_parameters_deep(layer_dims):
    # dictionary 객체 생성
    parameters = {}
    # 총 layer들의 길이를 계산
    L = len(layer_dims)
    # 레이어들을 돌면서, 레이어들 간의 weight와 bias의 초기값의 난수 생성
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1]) #*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        # assert를 통해, dimension을 맞추줍니다. 틀릴시 error 발생
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))
        
    return parameters

In [2]:
parameters = initialize_parameters_deep(layer_dims)
parameters 

{'W1': array([[ 0.96611671, -0.29382973, -0.39793937, -0.39615487, -0.01055339],
        [-0.48097877, -0.03133202,  0.68878007,  0.25151511,  0.16290848],
        [ 0.15159429, -0.15188782, -0.31257492,  0.50734366, -0.54890941],
        [ 0.13220054, -0.41171213,  0.3657522 , -0.05180709, -0.36550746]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.]]),
 'W2': array([[ 0.37020287,  0.00858161, -0.65042959,  0.58061724],
        [-0.07171643, -0.09504507,  0.11313475, -0.58871746],
        [-0.17561212,  0.01274369, -0.07963832, -0.18083417]]),
 'b2': array([[0.],
        [0.],
        [0.]]),
 'W3': array([[ 1.19715607, -0.1820765 ,  0.29266063]]),
 'b3': array([[0.]])}

In [3]:
def linear_forward(A, W, b):
    # W에 A를 내적하게 됩니다. 그후에는 b를 더해줍니다.
    Z = W.dot(A) + b
    # Z의 shape이 input과 weight의 shape과 동일한지를 체크합니다.
    assert(Z.shape == (W.shape[0], A.shape[1]))
    # 계산단계에서 사용한 값을 cache에 저장해둡니다.
    cache = (A, W, b)
    return Z, cache

def linear_activation_forward(A_prev, W, b, activation):
    # Activation function의 종류에 따라서 값을 나누어 줍니다.
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    
    # Shape이 input과 weight와 동일한지 체크해줍니다.
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    # linear 연산과 activation 연산을 cache에 저장해둡니다.
    cache = (linear_cache, activation_cache)
    return A, cache

In [4]:
def L_model_forward(X, parameters):
    # cache 들의 list입니다.
    caches = []
    A = X
    # weight와 bias가 저장되어 있기 때문에 //2 를 해주어야 layer의 사이즈가 됩니다.
    L = len(parameters) // 2
    
    # hidden layersms relu를 통과
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "relu")
        caches.append(cache)
    
    # output layer는 sigmoid를 통과하게 한다
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], activation = "sigmoid")
    caches.append(cache)
    assert(AL.shape == (1,X.shape[1]))
    return AL, caches

In [6]:
X = np.random.randn(5,4)
Y = np.array([[0, 1, 1, 0]])
AL, caches = L_model_forward(X, parameters)
AL

array([[0.50101001, 0.49254327, 0.60241226, 0.5029799 ]])

In [9]:
# Cost Function
# Arguments - AL : 뉴럴넷 통과하여 나온 y, Y : 실제 label vector
# Returns - cost : cross-entropy cost

In [7]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = (-1.0/m)*np.sum(np.multiply(Y,np.log(AL)) + np.multiply(1-Y, np.log(1-AL)))
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    return cost

In [8]:
cost = compute_cost(AL, Y)
print("cost = " + str(cost))

cost = 0.6523200689754123


In [10]:
# Backpropagation process
# LINEAR backward
# LINEAR -> ACTIVATION backward
# Layer -> Layer backward

def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    assert (dZ.shape == Z.shape)
    return dZ

def sigmoid_backward(dA, cache):
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    assert (dZ.shape == Z.shape)
    return dZ

def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = np.dot(dZ,cache[0].T)/m
    db = np.sum(dZ, axis=1, keepdims=True)/m
    dA_prev = np.dot(cache[1].T, dZ)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db =  linear_backward(dZ, linear_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    return dA_prev, dW, db

In [11]:
def L_model_backward(AL, Y, caches):
    grads = {} # 빈 dictionary 호출
    L = len(caches) # 레이어의 갯수를 caches로 부터 받아옵니다.
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # Shape을 AL과 동일하게 해줍니다.
    
    # Initializing the backpropagation
    dAL = - (np.divide(Y,AL)- np.divide(1-Y, 1-AL))
    # caches index를 잡아둡니다.
    current_cache = caches[L-1] 
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation="sigmoid")
    
    for l in reversed(range(L-1)):
        # indexing입니다.
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA"+str(l+1)], current_cache, activation="relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
    return grads

In [12]:
grads = L_model_backward(AL, Y, caches)
grads

{'dA2': array([[ 0.59978718, -0.60750491, -0.47597458,  0.60214545],
        [-0.09122215,  0.09239594,  0.07239138, -0.09158082],
        [ 0.1466259 , -0.14851261, -0.11635828,  0.14720241]]),
 'dW3': array([[-0.03315558, -0.02078383,  0.00135047]]),
 'db3': array([[0.02473636]]),
 'dA1': array([[ 0.19629365, -0.00662631, -0.17620715,  0.19706544],
        [ 0.0070157 , -0.00878178, -0.00408463,  0.00704328],
        [-0.40179637,  0.01045319,  0.30958795, -0.40337617],
        [ 0.3217318 , -0.0543951 , -0.27635905,  0.3229968 ]]),
 'dW2': array([[-0.0551107 ,  0.14156149,  0.        , -0.03531602],
        [ 0.        ,  0.0061461 ,  0.04558699,  0.00134032],
        [ 0.00026072,  0.03460658,  0.        ,  0.        ]]),
 'db2': array([[0.18148951],
        [0.02309899],
        [0.07345708]]),
 'dA0': array([[-0.0033744 , -0.00138257, -0.20677149,  0.18700055],
        [-0.00021982,  0.02108256,  0.16555527, -0.05812437],
        [ 0.00483227, -0.02921125, -0.03095917, -0.0735688

In [13]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2 # 레이어의 갯수입니다.
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*grads["dW"+str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*grads["db"+str(l+1)]
    return parameters

In [14]:
parameters = update_parameters(parameters, grads, 0.05)
parameters

{'W1': array([[ 0.96545114, -0.29611599, -0.39947123, -0.39699326, -0.01547299],
        [-0.48099997, -0.0316113 ,  0.6886039 ,  0.25193102,  0.16256599],
        [ 0.15166953, -0.15173056, -0.31244795,  0.50699012, -0.54886142],
        [ 0.13131985, -0.41617915,  0.36503494, -0.05255228, -0.36431937]]),
 'b1': array([[-2.60728589e-04],
        [-6.59649945e-05],
        [-1.30664895e-04],
        [ 4.13442692e-03]]),
 'W2': array([[ 0.3729584 ,  0.00150354, -0.65042959,  0.58238304],
        [-0.07171643, -0.09535238,  0.1108554 , -0.58878447],
        [-0.17562516,  0.01101337, -0.07963832, -0.18083417]]),
 'b2': array([[-0.00907448],
        [-0.00115495],
        [-0.00367285]]),
 'W3': array([[ 1.19881385, -0.1810373 ,  0.2925931 ]]),
 'b3': array([[-0.00123682]])}

In [16]:
#CNN : filter, pooling 
def zero_pad(X,pad):
    X_pad = np.pad(X,((0,0),(pad,pad),(pad,pad),(0,0)),'constant',constant_values=0)
    return X_pad

x = np.random.randn(4,3,3,2)
x_pad = zero_pad(x, 2)
print(x.shape, x_pad.shape)
x_pad

(4, 3, 3, 2) (4, 7, 7, 2)


array([[[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [-0.01296614,  0.00608499],
         [ 0.90796727,  0.05052092],
         [-0.18234052,  0.51504166],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 1.28240989,  0.84210731],
         [ 0.35115932,  0.17818944],
         [-0.85912262, -0.68244724],
         [ 0.        ,  0.      

In [17]:
def conv_single_step(a_slice_prev, W, b):
    # Element-wise product
    s = a_slice_prev * W
    # 채널을 기반으로 모두 더해줍니다.
    Z = np.sum(s)
    # Bias b를 더해줍니다.
    Z = Z + np.float(b)
    return Z

In [18]:
def conv_forward(A_prev, W, b, hparameters):
    # Input의 shpae을 정의합니다.
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    # filter의 shape을 정의합니다.
    (f,f,n_C_prev,n_C) = W.shape
    # input dictionary에서 받을 value 값입니다.
    stride = hparameters['stride']
    pad = hparameters['pad']
    
    # Conv의 output volumn을 정의해줍니다
    n_H = int(((n_H_prev - f + (2*pad)) / stride)+1)
    n_W = int(((n_W_prev - f + (2*pad)) / stride)+1)
    
    # output volumn을 initialize해줍시다
    Z = np.zeros((m, n_H, n_W, n_C))
    
    # Padding을 설정해줍니다.
    A_prev_pad = zero_pad(A_prev,pad)
    
    for i in range(m): #batch에 있는 traindata를 조회
        a_prev_pad = A_prev_pad[i]
        for h in range(n_H): #hight를 돌고
            for w in range(n_W): #width를 돌고
                for c in range(n_C): #Channel을 돌면서
                    #input의 slice를 해줍시다
                    vert_start = h*stride
                    vert_end = vert_start+f
                    horiz_start = w*stride
                    horiz_end = horiz_start+f
                    
                    a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end,:]
                    Z[i,h,w,c]=conv_single_step(a_slice_prev,W[...,c],b[...,c])
    assert(Z.shape == (m,n_H,n_W,n_C))
    cache = (A_prev,W,b,hparameters)
    
    return Z, cache

In [19]:
np.random.seed(1)
A_prev = np.random.randn(10,4,4,3)
W = np.random.randn(2,2,3,8)
b = np.random.randn(1,1,1,8)
hparameters = {"pad" : 2,
               "stride": 2}
Z, cache_conv = conv_forward(A_prev, W, b, hparameters)

print("Z's mean =", np.mean(Z))

Z's mean = 0.048995203528855794


C:\Users\devtae\AppData\Local\Temp/ipykernel_3880/664427259.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Z = Z + np.float(b)


In [20]:
def pool_forward(A_prev, hparameters, mode="max"):
    # input의 shape을 받아옵니다
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    # filter size와 stride size를 받아옵니다.
    f = hparameters["f"]
    stride = hparameters["stride"]
    # ouput dimension을 잡아줍시다
    n_H = int(1+(n_H_prev-f)/stride)
    n_W = int(1+(n_W_prev-f)/stride)
    n_C = n_C_prev
    
    A = np.zeros((m, n_H, n_W, n_C))
    
    for i in range(m):
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h*stride
                    vert_end = vert_start + f
                    horiz_start = w*stride
                    horiz_end = horiz_start + f
                    
                    a_prev_slice = A_prev[i, vert_start:vert_end, horiz_start:horiz_end, c]
                    
                    if mode == "max":
                        A[i,h,w,c] = np.max(a_prev_slice)
                    elif mode == "average":
                        A[i,h,w,c] = np.mean(a_prev_slice)
                        
    cache = (A_prev, hparameters)
    assert(A.shape ==(m,n_H,n_W,n_C))
    
    return A, cache

In [21]:
np.random.seed(1)
A_prev = np.random.randn(2, 4, 4, 3)
hparameters = {"stride" : 2, "f": 3}
A, cache = pool_forward(A_prev, hparameters)
print(A)
A, cache = pool_forward(A_prev, hparameters, mode = "average")
print(A)

[[[[1.74481176 0.86540763 1.13376944]]]


 [[[1.13162939 1.51981682 2.18557541]]]]
[[[[ 0.02105773 -0.20328806 -0.40389855]]]


 [[[-0.22154621  0.51716526  0.48155844]]]]


In [ ]:
#RNN

